In [202]:
# Points Accrual and Storage System
# Make code python 2 and 3 compliant
from __future__ import print_function

import datetime
from dateutil import relativedelta
import pandas as pd

# Make things look pretty
from pprint import pprint
import sqlite3

In [203]:
class Db(object):    
    """The database class."""

    def __init__(self, database='fenix.db', statements=[]):
        """Initialize a new database or connect to an existing one."""

        self.database = database  #the database filename
        self.statement = ''       #holds incomplete statements
        
        self.connect()
        self.execute(statements)
        self.close()            

    def connect(self):
        """Connect to the database."""

        self.connection = sqlite3.connect(self.database)
        self.cursor = self.connection.cursor()
        self.connected = True
        self.statement = ''

    def close(self): 
        """Close the database."""

        self.connection.commit()
        self.connection.close()
        self.connected = False

    def incomplete(self, statement):
        """Concatenate clauses until a complete statement is made."""

        self.statement += statement
        if self.statement.count(';') > 1:
            self.statement = ''
            raise ValueError('One statement at a time, please!.')
            
        return not sqlite3.complete_statement(self.statement)
    
    def execute(self, statements):
        """Executes SQL statements. """

        queries = []
        close = False
        if not self.connected:                  
            self.connect() 
            close = True
        if type(statements) == str:
            statements = [statements]
        for statement in statements:
            if self.incomplete(statement):
                continue #the statement is complete
            try:
                statement = self.statement.strip() #reset the test statement
                self.statement = ''
                self.cursor.execute(statement) # Execute statement
                data = self.cursor.fetchall()
                if statement.upper().startswith('SELECT'):
                    queries.append(data)

            except sqlite3.Error as error:
                raise ValueError('An error occurred:', error.args[0])

        if close:
            self.close()   #Close database if opened here.
        
        if queries:
            return queries
        
        
    def from_csv(self,table,filename):
        """Imports data from a csv file to a selected table."""
        
        if not self.connected:                  
            self.connect() 
            close = True
            
        data = pd.read_csv(filename)
        data.to_sql(table,self.connection,if_exists='append',index=False)
        
        if close:
            self.close() 

In [204]:
class Employee(object):
    """Defines an employee's required attributes."""
    
    def __init__(self, ID):
        """
            Create an instance of an employee.
            
            Arguments
            id - <int> - This is the employee's ID.
        """
        self.id = ID
        
    def get_id(self):
        return self.id
    
    def get_attribute(self,attribute):
        """Returns the selected attributed value."""
                  
        statement = ['SELECT {} FROM employees WHERE emp_id={};'.format(attribute, self.get_id())]
        attribute = db.execute(statement)[0][0][0] # What?!
        return attribute
    
    
    def request(self,amount,signoff=False):
        """Allows an employee to request points withdrawal."""
        
        # Check points balance
        statement = ['SELECT points FROM employees WHERE emp_id={};'.format(self.get_id())]
        points_balance = db.execute(statement)[0][0][0]
        
        if (points_balance > amount):
            if signoff:
                # Send expenses request to finance
                statement = ['INSERT INTO requests (emp_id,withdrawal) VALUES ({},{});'.format(self.get_id(),amount)]
                db.execute(statement)
                req_id = db.execute(['SELECT req_id FROM requests WHERE emp_id={} and withdrawal={} and status=0;'.format(self.get_id(),amount)])
                return req_id[0][0][0]
            else:
                print('Manager sign off required.')
        else:
            raise ValueError('Insufficient Points.')

In [209]:
class Accrual(object):
    """This class performs the points accrual and tracking."""
    
    def __init__(self):
        self.today = f"{datetime.date.today():%Y/%m/%d}"
        
        
    def points_multiplier(self,points,tenure):
        """Multiples the points as per the tenure"""
        
        #points = 0;
        if (tenure > 0):
            if   (0.0 < tenure < 2.0):
                points *= 1.0
            elif (2.0 <= tenure < 4.0):
                points *= 1.25
            else:
                points *= 1.5
        return points
            
    def seniority_points(self, seniority):
        """Returns the number of points per month given the seniority."""
        
        points =  None
        
        if seniority   == 'A':
            points=5
        elif seniority == 'B':
            points=10
        elif seniority == 'C':
            points=15
        elif seniority == 'D':
            points=20
        elif seniority == 'E':
            points=25
        else:
            points = 0
            
        return points
    
    def time_difference(self,start_date,end_date):
        """
            Returns the number of months
            
            Arguments
            start_date - <string> - The date of reference at the start of the calculation period.
            end_date - <string> - The date of reference at the end of the calculation period
        """
        
        start_date = datetime.datetime.strptime(str(start_date), '%Y/%m/%d')
        end_date = datetime.datetime.strptime(str(end_date), '%Y/%m/%d')
        months = (end_date.year - start_date.year) * 12 + (end_date.month - start_date.month)
        return months
        
    def points_accrual(self,ID):
        """
            Calculates the number of points for a given employee as per today.
            
            Arguments
            id - <int> - The employee id
        """
        
        statement = ['SELECT start_date FROM history where emp_id={};'.format(ID)]
        results = db.execute(statement)[0]
        
        dates = []      # Unpack dates.
        for date in results:
            dates.append(date[0])
            
        dates.sort(key=lambda date: datetime.datetime.strptime(date,'%Y/%m/%d'))
        
        n = 0;
        points = [];
        
        while(n < (len(dates)-1)):
            
            months = self.time_difference(dates[n],dates[n+1])
            temp = self.calculator_helper(dates[n],ID, months)
            points.append(temp)
            n = n+1
            
        # Points up to today.
        months = self.time_difference(dates[-1],self.today)
        temp = self.calculator_helper(dates[-1],ID,months)
        points.append(temp)
        
        points = sum(points) # Sum up the points (All Points)
        
        # Current points = All points - Used points
        statement = ['SELECT withdrawal FROM requests where emp_id={} and status={};'.format(ID,1)]
        used_points = db.execute(statement)[0]
        if used_points:
            points = points - used_points
            
        # Update employees table.
        statement = ['UPDATE employees SET points={} WHERE emp_id={};'.format(points,ID)]
        db.execute(statement)
            
        return points
    
    def calculator_helper(self,date,ID,months):
        """Calculates points per month"""
    
        statement = ['SELECT seniority FROM history where start_date=\'{}\' and emp_id={};'.format(date,ID)]
        seniority = db.execute(statement)[0][0][0]
        sen_points = self.seniority_points(seniority)
        
        points_per_month = sen_points*months
        
        # Calculate tenure
        tenure = months/12
        temp = self.points_multiplier(points_per_month,tenure)
        
        return temp
    
    def balance_deduction(self,ID,req_id,approved=False):
        """Deducts the used points, and updates the points balance"""
        
        if approved:
            statement = ['SELECT withdrawal FROM requests where emp_id={} and req_id={};'.format(ID,req_id)]
            deduction = db.execute(statement)[0][0][0]
            
            statement = ['SELECT points FROM employees WHERE emp_id={};'.format(ID)]
            points = db.execute(statement)[0][0][0]
            
            print(points)
            print(deduction)
            
            new_points = points - deduction
            # Update employees table.
            statement = ['UPDATE employees SET points={} WHERE emp_id={};'.format(new_points,ID),
                        'UPDATE requests SET status=1 WHERE emp_id={} and req_id={};'.format(ID,req_id)]
            db.execute(statement)
            
            # Expense request creation sent to finance

In [210]:
# Create tables(run this only once)
a = ('create table if not exists employees(emp_id INTEGER PRIMARY KEY, seniority TEXT,start_date DATE, points FLOAT DEFAULT 0.0 NOT NULL);') 
b = ('create table if not exists history(hist_id INTEGER PRIMARY KEY,emp_id INTEGER, start_date DATE,seniority TEXT, FOREIGN KEY(emp_id) REFERENCES employees (emp_id) ON DELETE CASCADE ON UPDATE NO ACTION);')
c = ('create table if not exists requests(req_id INTEGER PRIMARY KEY,emp_id INTEGER, withdrawal FLOAT,status INTEGER DEFAULT 0,FOREIGN KEY(emp_id) REFERENCES employees (emp_id) ON DELETE CASCADE ON UPDATE NO ACTION);')

statements = [a,b,c]
db = Db('Fenix.db',statements)

In [211]:
db = Db('Fenix.db')

In [212]:
db.from_csv('history','History.csv') # Reading from a csv file

In [213]:
db.from_csv('employees','Employee.csv')

In [214]:
statement = ['SELECT * FROM employees;']
results = db.execute(statement)[0]
results

[(1, 'D', '2016/01/02', 0.0),
 (2, 'C', '2015/12/01', 0.0),
 (3, 'B', '2017/04/02', 0.0),
 (4, 'A', '2015/05/27', 0.0),
 (5, 'A', '2019/03/14', 0.0)]

In [233]:
statement = ['SELECT withdrawal FROM requests where emp_id=1 and status=1;']
results = db.execute(statement)[0]
results 

[]

In [215]:
acc = Accrual()

In [171]:
emp= Employee(2)

In [236]:
# Allow an employee to privately query their own points balance (allow for access controlled balance requests)
result = emp.get_id()
result

2

In [237]:
acc.points_accrual(5)

15.0

In [238]:
statement = ['SELECT * FROM employees;']
results = db.execute(statement)
results

[[(1, 'D', '2016/01/02', 750.0),
  (2, 'C', '2015/12/01', 532.5),
  (3, 'B', '2017/04/02', 325.0),
  (4, 'A', '2015/05/27', 775.0),
  (5, 'A', '2019/03/14', 15.0)]]

In [239]:
emp2 = Employee(2)

In [240]:
emp2.get_attribute('points')

532.5

In [173]:
statement = ['SELECT * FROM requests;']
results = db.execute(statement)
results

[[(1, 2, 10000.0, 0), (2, 2, 10000.0, 0), (3, 2, 10000.0, 0)]]

In [225]:
emp2.request(10000,True) # Throws a ValueError if the points request is more than points an employee has.

ValueError: Insufficient Points.

In [241]:
statement = ['SELECT * FROM employees;']
results = db.execute(statement)
results

[[(1, 'D', '2016/01/02', 750.0),
  (2, 'C', '2015/12/01', 532.5),
  (3, 'B', '2017/04/02', 325.0),
  (4, 'A', '2015/05/27', 775.0),
  (5, 'A', '2019/03/14', 15.0)]]

In [226]:
req_id = emp2.request(10,True)

In [227]:
req_id

1

In [228]:
acc.balance_deduction(2,req_id)

In [232]:
statement = ['SELECT * FROM requests;']
results = db.execute(statement)
results

[[(1, 2, 10.0, 1)]]

In [229]:
acc.balance_deduction(2,req_id,True)

542.5
10.0
